# ML Pipeline Preparation
For the machine learning portion, you will split the data into a training set and a test set. Then, you will create a machine learning pipeline that uses NLTK, as well as scikit-learn's Pipeline and GridSearchCV to output a final model that uses the message column to predict classifications for 36 categories (multi-output classification). Finally, you will export your model to a pickle file. After completing the notebook, you'll need to include your final machine learning code in train_classifier.py.

Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [16]:
# import libraries
import pandas as pd
pd.set_option('display.max_colwidth', 500)
from sqlalchemy import create_engine
import sqlite3
import re

In [17]:
import sqlite3
import pandas as pd

# connect to the database
conn = sqlite3.connect('../data/DisasterResponse.db')

# run a query
df = pd.read_sql('SELECT * FROM disaster_response_messages', conn)

In [18]:
df.shape

(26216, 40)

In [19]:
df.head(1)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that could pass over Haiti,Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
df.shape

(26216, 40)

In [21]:
df.columns[4:]

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [22]:
# def load_data():
#     df = pd.read_csv('corporate_messaging.csv', encoding='latin-1')
#     df = df[(df["category:confidence"] == 1) & (df['category'] != 'Exclude')]
#     X = df.text.values
#     y = df.category.values
#     return X, y

In [23]:
y = df[df.columns[4:]].values
X = df['message'].values

### 2. Write a tokenization function to process your text data

In [24]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download(['punkt', 'wordnet','stopwords'])

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/henriettewevell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/henriettewevell/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/henriettewevell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# # download necessary NLTK data
# import nltk
# nltk.download(['punkt', 'wordnet'])
# from nltk.stem.wordnet import WordNetLemmatizer

In [26]:
def replace_all_urls(text):
        url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        detected_urls = re.findall(url_regex, text)
        for url in detected_urls:
             text = text.replace(url, "urlplaceholder")
        return text

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    test = text.strip() # remove whitespaces
    return test

def tokenize(text):
    words = word_tokenize(text)
    return words

def remove_stopwords(text):
    stop_words = stopwords.words("english")
    words = [w for w in text if w not in stop_words]
    return words

def lemmatize(text: list):
    lemmatized_all = []
    for word in text:
        lemmatizer = WordNetLemmatizer()
        lemmatized = lemmatizer.lemmatize(word, pos='v')
        lemmatized_all.append(lemmatized)
    return lemmatized_all


def clean_and_tokenize(text: str) -> list:
    text = replace_all_urls(text)
    text = clean_text(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = lemmatize(tokens)
    return tokens

# def tokenize(text):
#     detected_urls = re.findall(url_regex, text)
#     for url in detected_urls:
#         text = text.replace(url, "urlplaceholder")

#     tokens = word_tokenize(text)
#     lemmatizer = WordNetLemmatizer()

#     clean_tokens = []
#     for tok in tokens:
#         clean_tok = lemmatizer.lemmatize(tok).lower().strip()
#         clean_tokens.append(clean_tok)

#     return clean_tokens


In [38]:
clean_and_tokenize("If you want to find a Job at an NGO or the Government, upload your resume at http://www.jobpaw.com/")

['want',
 'find',
 'job',
 'ngo',
 'government',
 'upload',
 'resume',
 'urlplaceholder']

In [28]:
# url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
# urls_df = df[df['message'].str.contains(url_regex)]['message'].head(1)
# urls_df

In [29]:
# dilter the df messages column for when url is present
# urls_df = df[df['message'].str.contains(url_regex)]['message'].head(5)
# urls_df
# number_regex = r'\d+,\d+'
# # or number.number
# number_regex2 = r'\d+\.\d+'
# # or number
# number_regex3 = r'\d+'

# df[df['message'].str.contains(number_regex2)]['message'].sample(n=15)

# filter the df messages column for number_regex or number_regex2 or number_regex3 is present
# df[df['message'].str.contains(number_regex) | df['message'].str.contains(number_regex2) | df['message'].str.contains(number_regex3)]['message'].sample(n=15)

In [30]:
# tokenize(df.iloc[10]['message'])

In [31]:
# number_and_letters_regex = r'\d+[a-zA-Z]+'
# letters_and_number_regex = r'[a-zA-Z]+\d+'
# combined_regex = r'(\d+[a-zA-Z]+)|([a-zA-Z]+\d+)'
# # df[df['message'].str.contains(number_and_letters_regex)]['message'].sample(n=15)

# # get number_and_letters_regex from each message
# df[df['message'].str.contains(letters_and_number_regex)]['message'].str.findall(letters_and_number_regex).head()

In [32]:
# df[df['message'].str.contains(number_and_letters_regex)]['message'].str.findall(number_and_letters_regex).head()

In [33]:
# df[df['message'].str.contains(combined_regex)]['message'].str.findall(combined_regex).head()

In [34]:
# filter df messages for when it contains any numbers
# df[df['message'].str.contains('\d')]['message'].sample(n=10)

In [35]:
# remove stopwords
# lower case
# lemmatize?
# remove urls
# remove punctuation
# remove numbers?

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [ ]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)


def main():
    # load data and perform train text split
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # instantiate transformers and classifiers
    vect = CountVectorizer(tokenizer=tokenize)
    tfidf = TfidfTransformer()
    clf = RandomForestClassifier()

    # fit and transform the training data
    X_train_counts = vect.fit_transform(X_train)
    X_train_tfidf = tfidf.fit_transform(X_train_counts)

    # train classifier
    clf.fit(X_train_tfidf, y_train)

    # transform (no fitting) the test data
    X_test_counts = vect.transform(X_test)
    X_test_tfidf = tfidf.transform(X_test_counts)
    # predict on test data
    y_pred = clf.predict(X_test_tfidf)

    # display results
    display_results(y_test, y_pred)

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
clf =   pipeline.fit(X_train, y_train)

/Users/henriettewevell/src/udacity_disaster_response/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# predict on test data
y_pred = pipeline.predict(X_test)
y_pred
# display results
# display_results(y_test, y_pred)

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

SyntaxError: invalid syntax (3277122076.py, line 1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.